# Final Project: Phase 2

---

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import json
import warnings
warnings.filterwarnings('ignore')

## Reimporting

First, let's reimport all of the data we finalized during our EDA session.

In [7]:
df = pd.read_csv("./project_data/USVideos.csv")

with open("./project_data/US_category_id.json") as f:
    categories = json.load(f)

cat_map = {}
for index,cat in enumerate(categories["items"]):
    cat_map[int(cat["id"])]=cat["snippet"]["title"]
    
df["category"] = df["category_id"].map(cat_map)

data = df[["title","channel_title","category","description","tags","views"]]

data["log_views"] = np.log(data["views"]+1)

FileNotFoundError: [Errno 2] No such file or directory: './project_data/USVideos.csv'

Let's take stock of what we have:

In [ ]:
data.head()

## Splitting into Train/Test

First, we split our dataset into features and class variables.

In [ ]:
X = data.drop("views",axis=1)
Y = data["log_views"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, Y, random_state=42)

In [ ]:
# Checking to see if they're together
X["channel_title"].value_counts().head()

In [ ]:
X_train_raw["channel_title"].value_counts().head()

In [ ]:
X_test_raw["channel_title"].value_counts().head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

## Feature Transformation

Use your training data to fit any transformers or encoder your need, then apply the fit transformer to your test data. This applies to:
* Normalizing/standardizing your features
* Using Bag of Words or TF-IDF to encode strings
* PCA or dimensionality reduction

**Rationale**: In practice, we won't be able to see the test data we'll be making predicting for, so we shouldn't use that data as the basis for any transformation or feature extraction.

Here, we **only take the top N** words when doing our count vectors. We do this because to use the full word list would be infeasable since there are so many. Also, due to the nature of language, there are probably only a few words that are used frequently - but many words used only once.

In [ ]:
# This function applies our feature transformation to our training and test data
# top_n

def apply_feature_transformation(X_train, X_test,top_n=100):
    # First, create and fit our vectorizers on the training data
    # We only take the top_n words into consideration, and simply ignore anything outside of that
    tag_vectorizer = CountVectorizer(token_pattern=r"([^|]+)", max_features=top_n)
    tag_vectors = tag_vectorizer.fit_transform(X_train["tags"].str.replace("\"","")).toarray()

    description_vectorizer = CountVectorizer(max_features=top_n)
    description_vectors = description_vectorizer.fit_transform(X_train["description"].values.astype('U')).toarray()

    title_vectorizer = CountVectorizer(max_features=top_n)
    title_vectors = title_vectorizer.fit_transform(X_train["title"]).toarray()

    # Next, we concatinate the tag, description, and title vectors into one vector
    # This is so we can have one "object" that we can fit into our models
    # Therefore, this will create a matrix of size [num_train_instances * (3 * top_n)]
    X_train_transfomed = np.concatenate((tag_vectors, description_vectors, title_vectors), axis=1)
    
    # Now we fit our test data
    # This will create a matrix of size [num_test_instances * (3 * top_n)]
    X_test_transformed = np.concatenate(
        (
            tag_vectorizer.transform(X_test["tags"].str.replace("\"","")).toarray(),
            description_vectorizer.transform(X_test["description"].values.astype('U')).toarray(),
            title_vectorizer.transform(X_test["title"]).toarray()
        ),
        axis=1
    )
    
    return X_train_transfomed, X_test_transformed

In [ ]:
X_train, X_test = apply_feature_transformation(X_train_raw,X_test_raw,100)

In [ ]:
# Checking to make sure we shaped our features right
# There should be 3*top_n features
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# Trying Different Models

Now that we have our data fully preprocessed, let's try it with some different models to see what we can get.

### Linear Regression

It's always good to work your way up from the simplest models you know before trying the more complicated ones. So let's see if a simple linear regression will solve our problem.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
print("Training score", reg.score(X_train,y_train))
print("Testing score:", reg.score(X_test,y_test))

Eugh. Looks like linear regression isn't powerful enough for this problem. There's no use trying to tune this model if even the base accuracy doesn't look promising.

### KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train,y_train)

Evaluating the KNN Regression takes some time, so we'll only evaluate the testing set.

In [ ]:
print("Testing score:", neigh.score(X_test,y_test))

### Multi Layer Perceptron Regression

Let's see if we can squeeze anything out of a neural network. (What you didn't see was fiddling with the params and layer topology)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
nn = MLPRegressor(max_iter=45, hidden_layer_sizes = (300,100,50,10))
nn.fit(X_train, y_train)

In [ ]:
nn.score(X_train,y_train)

In [ ]:
nn.score(X_test,y_test)

**To continue, let's choose the KNN model**. This model is semi-interpretable, so it's perfered over the ANN.

## Hyperparameter Tuning
For the sake of time, we'll only be finding hyperparams for our KNN model.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def find_best_hyperparameters(X_train, y_train):
    """
    Input: The training X features and Y labels/values
    Output: The classifier with the best hyperparams, the predictions
    """
    neigh = KNeighborsRegressor()
    param_grid = {"n_neighbors": [1,2],
                  "p": [1,2]}
    
    # Warning, takes a while!
    search = GridSearchCV(neigh, param_grid)
    search.fit(X_train,y_train)
    return search, search.predict(X_test)

In [ ]:
best_model, predictions = find_best_hyperparameters(X_train, y_train)

In [ ]:
best_model

In [ ]:
predictions